In [15]:
import os
import sys
import pandas as pd
from shutil import copy
from openpyxl import load_workbook
from adding_an_addition_following_the_index import adding_an_addition
from functions import change_Muni_Heb_to_Muni_Eng
from division_into_traffic_zones_of_plans import division_into_traffic_zones
from export_index_layer_for_client_control import export_index_layer
from index_layer import index_layer_fun
from status_exists_for_control import export_status_exists
from uploading_index_table_elements import uploading_index_table
from uploading_index_table_elements import uploading_index_table
from export_forecast_2040 import export_forecast

In [16]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [17]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [18]:
cwd = os.getcwd()
software_data_folder_location = os.path.dirname(cwd)
software_data_folder_location
df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

In [19]:
create_forecast_basic_location=df_inputs_outputs['location'][0]
forecast_version_basic_folder_location=df_inputs_outputs['location'][4]
client_data_folder_location=df_inputs_outputs['location'][1]
forecast_version=df_inputs_outputs['location'][2]
v_date=df_inputs_outputs['location'][3]
index_file_name='index_format_for_creating_forecast_jtmt_input_{}_{}'.format(forecast_version,v_date)

In [20]:
from functions import up_load_gdb, make_point, find_files_with_pattern, up_load_shp, up_load_df, split_index_by_taz

In [21]:
# העלת אזורי תנועה לחישוב
folder_path=r'{}\For_approval\Reference_tabels\shp'.format(client_data_folder_location)

gpd_name='tochnit_check.gdb'

forecast = up_load_gdb(r'{}\{}'.format(folder_path,gpd_name),'TAZ_211028_V3_Published_with_client_changes')

In [22]:
#### הוספת מאפיינים גיאוגרפים לאזורי תנועה
forecast_point = make_point(forecast)

    # Load data layers
DISTRICT = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_DISTRICT')
urban = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_urban')
SCHOOLDISTRICT = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_SCHOOLDISTRICT')
PUMA = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_PUMA')
jerusalem_city = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_In_jerusal')
subdistrict_il = up_load_gdb(
        r'{}\background_files\subdistrict2008.gdb'.format(software_data_folder_location), 'subdistrict2008_ITM')
muni_JTMT = up_load_gdb(
        r'{}\background_files\MUNI_border.gdb'.format(software_data_folder_location), 'muni_under_JTMT_ITM')
    
# מחפש קבצים של jeru_metro_jtmt_border
jeru_metro_jtmt_border_files = find_files_with_pattern(r'{}\background_files'.format(software_data_folder_location), 'jeru_metro_jtmt_border')
filepath = ''

# לוקח את הקובץ של השכבה
for string in jeru_metro_jtmt_border_files:
    if string.endswith('.shp'):
        filepath = string

# מעלה את השכבה
jeru_metro_jtmt_border = up_load_shp(filepath)

# Geographical join between traffic zones and data layers
forecast_point_DISTRICT = forecast_point.sjoin(
        DISTRICT)[['Taz_num', 'puma2040_csv_DISTRICT']]
forecast_point_urban = forecast_point.sjoin(
        urban)[['Taz_num', 'BaseProjections2040_csv_urban']]
forecast_point_SCHOOLDISTRICT = forecast_point.sjoin(
        SCHOOLDISTRICT)[['Taz_num', 'puma2040_csv_SCHOOLDISTRICT']]
forecast_point_PUMA = forecast_point.sjoin(
        PUMA)[['Taz_num', 'puma2040_csv_PUMA']]
forecast_point_jerusalem_city = forecast_point.sjoin(
        jerusalem_city)[['Taz_num', 'jerusalem_city']]
forecast_point_subdistrict_il = forecast_point.sjoin(
        subdistrict_il[['geometry', 'ENG_NAME_nafa']])[['Taz_num', 'ENG_NAME_nafa']]
forecast_point_muni_JTMT = forecast_point.query('main_sector!="Palestinian"').sjoin(
        muni_JTMT[['Muni_Heb', 'Sug_Muni', 'CR_PNIM', 'geometry']])[['Taz_num', 'Muni_Heb', 'Sug_Muni', 'CR_PNIM']]
forecast_point_jeru_metro_jtmt_border = forecast_point.sjoin(
        jeru_metro_jtmt_border)[['Taz_num', 'jeru_metro']]

# Merge tables into one table
forecast = (forecast
                .merge(forecast_point_subdistrict_il, on='Taz_num', how='left')
                .merge(forecast_point_muni_JTMT, on='Taz_num', how='left')
                .merge(forecast_point_jeru_metro_jtmt_border, on='Taz_num', how='left')
                .merge(forecast_point_DISTRICT, on='Taz_num', how='left')
                .merge(forecast_point_urban, on='Taz_num', how='left')
                .merge(forecast_point_SCHOOLDISTRICT, on='Taz_num', how='left')
                .merge(forecast_point_PUMA, on='Taz_num', how='left')
                .merge(forecast_point_jerusalem_city, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names
forecast.rename(columns={'ENG_NAME_nafa': 'zonetype'}, inplace=True)
forecast.rename(columns={'puma2040_csv_DISTRICT': 'DISTRICT'}, inplace=True)
forecast.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)
forecast.rename(columns={'puma2040_csv_PUMA': 'PUMA'}, inplace=True)
forecast.rename(columns={'puma2040_csv_SCHOOLDISTRICT': 'SCHOOLDISTRICT'}, inplace=True)


# Data processing for feature columns
forecast.loc[forecast['main_sector'] == 'Palestinian', 'zonetype'] = 'Palestinian'
forecast['in_jerusalem_metropolin'] = 1
forecast.loc[forecast['jeru_metro'] == 0, 'in_jerusalem_metropolin'] = 0
forecast['yosh'] = 0
forecast.loc[forecast['zonetype'] == 'Judea and Samaria', 'yosh'] = 1

# Add constant columns
forecast['REGION'] = 1
forecast['slop'] = 0

forecast = forecast.set_index('Taz_num')
forecast['Taz_num'] = forecast.index

In [23]:
#### שינוי עמודה Muni_Heb
forecast=change_Muni_Heb_to_Muni_Eng(software_data_folder_location, forecast)

In [24]:
#### ייצוא שכבת אזורי תנועה לבקרת לקוח
save_shp_path=r'{}\For_approval\{}_taz_for_approval.shp'.format(client_data_folder_location,file_date)
    
col=['Taz_num','Name_hebre','Muni_Eng', 'main_sector', 'classification_name','Student_by_Classrooms','geometry']

forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')

C:\Users\dpere\AppData\Local\Temp\ipykernel_29568\938773556.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')


In [25]:
#### מצב קיים לבקרה
sys.path.insert(0, r'{}'.format(create_forecast_basic_location))
from run_create_forecast_basic import run_notebook

folder_path=r'{}\For_approval\Reference_tabels'.format(client_data_folder_location)
pattern='TAZ_V'
matching_files=find_files_with_pattern(r'{}\shp'.format(folder_path), pattern)

forecast_2020=None

# אם יש שכבות חדשות
if len(matching_files) > 0:
        suffix = '.shp'
        filtered_files_strings = []
        for string in matching_files:
            if string.endswith(suffix):
                filtered_files_strings.append(string)

        filepath=filtered_files_strings[0]

        #load excel file
        workbook = load_workbook(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))

        #open workbook
        sheet = workbook.active

        #modify the desired cell
        sheet["B4"] = folder_path
        sheet["B5"] = True
        sheet["B6"] = filepath

        #save the file
        workbook.save(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))
        # להריץ את קוד בסיס עם השכבות
        execution_result = run_notebook(r'{}\run_basic_from_ad_hoc.ipynb'.format(create_forecast_basic_location))
        print("ad-hoc-Notebook execution result:", execution_result)

        if execution_result == True:
            src_path = r'{}\forecast_2020_{}.xlsx'.format(folder_path, file_date)
            destination_path = r'{}\forecast_2020_{}_with_taz_changes.xlsx'.format(folder_path, file_date)

            copy(src_path, destination_path)
            os.remove(src_path)

            src_full_path = r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(folder_path, file_date)
            destination_full_path = r'{}\2020_jtmt_forcast_full_{}_with_taz_changes.xlsx'.format(folder_path, file_date)

            copy(src_full_path, destination_full_path)
            os.remove(src_full_path)

            forecast_2020=up_load_df(r'{}'.format(folder_path), r'2020_jtmt_forcast_full_{}_with_taz_changes'.format(file_date))
            #  forecast_2020=up_load_df(r'{}\background_files'.format(software_data_folder_location),'2020_jtmt_forcast_full_230720')

            col=[]

            col_20=['Taz_num','Taz_name',
                'main_secto',
                'aprt_20', 'pop_without_dorms_yeshiva',
                'student_toddlers',
                'student_gov',
                'cbs_muni_student_left_by_pre_of_demand_left',
                'uni_students', 'student_dorms',
                'emp_from_uni_student',
                'student_yeshiva',
                'emp_okev',
                'emp_not_okev','student']
                 
            forecast_2020=pd.merge(forecast[col].reset_index(),forecast_2020[col_20],how='left',on='Taz_num').fillna(0)

            save_excel_path=r'{}\For_approval\{}_forecast_2020_For_approval.xlsx'.format(client_data_folder_location,file_date)

            forecast_2020[col_20].to_excel(save_excel_path,index=False)
# אם אין שכבות חדשות
else:
    #load excel file
    workbook = load_workbook(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))

    #open workbook
    sheet = workbook.active

    #modify the desired cell
    sheet["B4"] = forecast_version_basic_folder_location
    sheet["B5"] = False
    sheet["B6"] = r'{}\background_files\TAZ_V4_230518_Published.shp'.format(create_forecast_basic_location)

    #save the file
    workbook.save(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))

    forecast_2020=up_load_df(r'{}\background_files'.format(software_data_folder_location),'2020_jtmt_forcast_full_230720')

    col=[]

    col_20=['Taz_num','Taz_name',
        'main_secto',
        'aprt_20', 'pop_without_dorms_yeshiva',
        'student_toddlers',
        'student_gov',
        'cbs_muni_student_left_by_pre_of_demand_left',
        'uni_students', 'student_dorms',
        'emp_from_uni_student',
        'student_yeshiva',
        'emp_okev',
        'emp_not_okev','student']

    forecast_2020=pd.merge(forecast[col].reset_index(),forecast_2020[col_20],how='left',on='Taz_num').fillna(0)

    save_excel_path=r'{}\For_approval\{}_forecast_2020_For_approval.xlsx'.format(client_data_folder_location,file_date)

    forecast_2020[col_20].to_excel(save_excel_path,index=False)

In [26]:
#### העלאת מרכיבי טבלת אינדקס
borders_index=up_load_shp(r'{}\For_approval\Reference_tabels\shp\gvul_index_with_ponten.shp'.format(client_data_folder_location))
index=up_load_df(r'{}\For_approval\Reference_tabels'.format(client_data_folder_location),index_file_name)
index=pd.merge(borders_index,index,on='id',how='right')

c:\Users\dpere\miniconda3\envs\creat_forecast_ad_hoc_env\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [27]:
## חלוקה לאזורי תנועה של התכניות
col=['add_uni_dorms',
    'add_old_age_home',
    'add_aprt',
    'Commerce_m2',
    'Business_m2',
    'Tourism_m2',
    'Public_m2',
    'Industry_m2',
    'emp_Public',
    'emp_Education',
    'emp_Commerce',
    'emp_Business',
    'emp_Industry',
    'emp_Tourism',
    'Classrooms',
    'add_uni_students']

index=split_index_by_taz(index,forecast,0.05,col)

In [29]:
### שכבת אינדקס
index=index.fillna(0)

promoteres_df = pd.read_excel(r'{}\promoteres.xlsx'.format(software_data_folder_location))


#מקדים לייצרת תעסוקה עוקב משקי בית
Industry_precent_per_hh=promoteres_df['value'][0]
Commerce_precent_per_hh=promoteres_df['value'][1]
Business_precent_per_hh=promoteres_df['value'][2]
Public_precent_per_hh=promoteres_df['value'][3]
Agriculture_precent_per_hh=promoteres_df['value'][4]

precent_emp_per_hh=promoteres_df['value'][5]

#מקדימים לייצרת מקומות עבודה מ"ר לפי ייעוד קרקע
m2_Industry_to_emp=promoteres_df['value'][6]
m2_Commerce_Hotel_to_emp=promoteres_df['value'][7]
m2_Business_to_emp=promoteres_df['value'][8]
m2_Public_to_emp=promoteres_df['value'][9]
m2_Agriculture_to_emp=promoteres_df['value'][10]
m2_Education_to_emp=promoteres_df['value'][11]
m2_Commerce_to_emp=m2_Commerce_Hotel_to_emp
m2_Tourism_to_emp=promoteres_df['value'][13]

#מילוי

old_age_home_fill=promoteres_df['value'][14]
uni_student_dorm_fill=promoteres_df['value'][15]


#מקדימי תעסוקה בעקבות חינוך


emp_education_per_student=promoteres_df['value'][16]
emp_Education_per_uni_student=promoteres_df['value'][17]
emp_Education_per_Yeshiva_student=promoteres_df['value'][18]


convert_dict={
'add_old_age_home': float,
'add_aprt': float,
'Commerce_m2': float,
'Business_m2': float,
'Tourism_m2': float,
'Public_m2': float,
'Industry_m2': float,
'emp_Public': float,
'emp_Education': float,
'emp_Commerce': float,
'emp_Business': float,
'emp_Industry': float,
'emp_Tourism': float,
'Classrooms': float,
'F2025': float,
'F2030': float,
'F2035': float,
'F2040': float,
'F2045': float,
'F2050': float,
'F2050_plus': float,
'Risk_factor': float,
'emp_fill_factor': float}

index = index.astype(convert_dict)

col_to_sum=['F2025',
'F2030',
'F2035',
'F2040']

index['precent_till_2040']=index[col_to_sum].sum(axis=1)

index['add_aprt_nominally']=index['add_aprt']

index['add_aprt']=index['add_aprt']*index['precent_till_2040']*index['Risk_factor']

index['Classrooms_nominally']=index['Classrooms']

index['Classrooms']=index['Classrooms']*index['precent_till_2040']*index['Risk_factor']

index['add_old_age_home_nominally']=index['add_old_age_home']

index['add_old_age_home']=index['add_old_age_home']*index['precent_till_2040']*index['Risk_factor']

index['add_uni_students_nominally']=index['add_uni_students']

index['add_uni_students']=index['add_uni_students']*index['precent_till_2040']*index['Risk_factor']

index['add_uni_dorms_nominally']=index['add_uni_dorms']

index['add_uni_dorms']=index['add_uni_dorms']*index['precent_till_2040']*index['Risk_factor']

list_category=['Commerce','Business','Industry','Tourism','Public']   #'Agriculture','Education','Public'
for c in list_category:
    index['{}_m2_nominally'.format(c)]=index['{}_m2'.format(c)]
    index['{}_m2'.format(c)]=index['{}_m2'.format(c)]*index['Risk_factor']*index['emp_fill_factor']*index['precent_till_2040']
    index['emp_{}_nominally'.format(c)]=index['emp_{}'.format(c)]
    index['emp_{}'.format(c)]=index['emp_{}'.format(c)]*index['precent_till_2040']*index['Risk_factor']*index['emp_fill_factor']
    index['add_emp_{}'.format(c)]=index['emp_{}'.format(c)]+index['{}_m2'.format(c)]/locals()['m2_{}_to_emp'.format(c)]

    col=['Taz_num','id',
    'add_aprt',
    'add_aprt_nominally',
    'add_old_age_home',
    'add_old_age_home_nominally',
    'add_uni_dorms',
    'add_uni_dorms_nominally',
    'add_uni_students',
    'add_uni_students_nominally',
    'Classrooms','Classrooms_nominally',
    'Commerce_m2',
    'Commerce_m2_nominally',
    'add_emp_Commerce',
    'Tourism_m2',
    'Tourism_m2_nominally',
    'add_emp_Tourism',
    'Business_m2',
    'Business_m2_nominally',
    'add_emp_Business',
    'Public_m2',
    'Public_m2_nominally',
    'add_emp_Public',
    'Industry_m2',
    'Industry_m2_nominally',
    'add_emp_Industry']
        
drop_geometry=index.drop(['geometry'], axis=1)

index=drop_geometry[col].pivot_table(index='Taz_num', aggfunc='sum').fillna(0)

In [ ]:
#### ייצוא שכבת אינדקס לבקרת לקוח
index_layer_for_client_control=export_index_layer(index_layer,client_data_folder_location,file_date,forecast_version)

### חישוב תחזית
#### הוספת תוספת בעקבות האינדקס
forecast=adding_an_addition(index_layer,forecast,forecast_2020,software_data_folder_location,client_data_folder_location,forecast_version)

#ייצוא תוצאות
export_forecast(forecast, client_data_folder_location, file_date, forecast_version,software_data_folder_location)

print('Done')